# Madrid Traffic Infrastructure Map Generator

This notebook creates an interactive map of Madrid's traffic infrastructure using multiple datasets. The map includes:
- Traffic lights
- CCTV cameras
- Street lamps
- Acoustic signals
- Intersections

Features:
- Toggle different infrastructure categories
- Filter by district with colored boundaries
- Real-time statistics
- Interactive popups with details
- Responsive design

In [1]:
import pandas as pd
import json
import os
from jinja2 import Template
import random

from shapely.geometry import shape, Point 

## 3. Data Loading Functions

### `clean_district_name(name)`
Standardizes district names to match GeoJSON formatting
- Removes extra spaces and hyphens
- Converts to title case

In [2]:
def clean_district_name(name):
    """Standardize district names to match GeoJSON formatting"""
    # Fix hyphens/spaces inconsistencies
    name = name.replace(' - ', '-').strip().title()
    
    # Handle special case for Moncloa
    if 'Moncloa' in name and 'Aravaca' not in name:
        return 'Moncloa-Aravaca'
    
    # Return standardized name
    return name

### `load_data(file_path, category)`
Loads and processes data files (CSV/Excel)
- Handles different file formats
- Cleans geographical data
- Converts to GeoJSON format

In [3]:
import pandas as pd
import json
import os
from shapely.geometry import shape, Point

# Cargar GeoJSON de distritos una sola vez
with open("madrid-districts.geojson") as f:
    districts_geojson = json.load(f)

# Crear estructura para búsqueda de distritos
district_polygons = []
for feature in districts_geojson['features']:
    polygon = shape(feature['geometry'])
    name = clean_district_name(feature['properties']['name'])
    district_polygons.append((name, polygon))

def get_district(lat, lon):
    """Obtener distrito usando coordenadas"""
    point = Point(lon, lat)
    for name, poly in district_polygons:
        if poly.contains(point):
            return name
    return 'Desconocido'

def load_data(file_path, category):
    """Cargar y procesar datos de infraestructura"""
    # Cargar CSV
    df = pd.read_csv(file_path)
    
    # Mapeo de columnas mejorado
    column_mapping = {
        'Latitude': 'latitude',
        'Longitude': 'longitude',
        'distrito': 'district',
        'tipo_elem': 'type',
        'id_cruce': 'crossing_id',
        'barrio': 'neighborhood',
        'direccion': 'address',
        'imagen': 'image_url'
    }
    
    # Renombrar columnas existentes
    df = df.rename(columns={k: v for k, v in column_mapping.items() if k in df.columns})
    
    # Limpiar coordenadas
    df = df.dropna(subset=['latitude', 'longitude'])
    df = df[(df['latitude'].between(40.2, 40.6)) & 
            (df['longitude'].between(-3.9, -3.5))]
    
    # Inferir distrito si no existe
    if 'district' not in df.columns:
        print(f"Infiriendo distritos para {category}...")
        df['district'] = df.apply(
            lambda row: get_district(row['latitude'], row['longitude']), 
            axis=1
        )
    
    # Limpieza final de nombres de distritos
    df['district'] = df['district'].apply(clean_district_name)
    
    # Convertir a GeoJSON
    features = []
    for _, row in df.iterrows():
        feature = {
            "type": "Feature",
            "geometry": {
                "type": "Point",
                "coordinates": [row['longitude'], row['latitude']]
            },
            "properties": {
                "category": category,
                "district": row['district'],
                **{col: str(row[col]) for col in df.columns 
                   if col not in ['latitude', 'longitude', 'district']}
            }
        }
        features.append(feature)
    
    return {
        "type": "FeatureCollection",
        "features": features
    }

# Función de limpieza de nombres actualizada
def clean_district_name(name):
    """Estandarizar nombres de distritos"""
    name = str(name).strip().title()
    replacements = {
        'Moncloa': 'Moncloa-Aravaca',
        'Fuencarral - El Pardo': 'Fuencarral-El Pardo',
        'Centro ': 'Centro'
    }
    return replacements.get(name, name)

## 4. Load and Process Data

Load all infrastructure datasets:
- Traffic Lights
- CCTV Cameras
- Streetlights
- Acoustic Signals
- Intersections

(Note: Update file paths as needed)

In [4]:
# Updated datasets loading without cruces and cctv
datasets = {
    "Traffic Lights": load_data("trafic.csv", "traffic-lights"),
    "Streetlights": load_data("lamps.csv", "streetlights"),
    "Acoustic Signals": load_data("acustic.csv", "acoustic-signals")
}

## 5. District Configuration

Load district boundaries and assign unique colors

In [5]:
# Cargar y preparar datos de distritos
with open("madrid-districts.geojson") as f:
    districts_geojson = json.load(f)

# Generar colores únicos y consistentes
district_colors = {}
districts = []

# Paleta de colores predefinida para mejor legibilidad
COLOR_PALETTE = [
    '#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd',
    '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf',
    '#aec7e8', '#ffbb78', '#98df8a', '#ff9896', '#c5b0d5',
    '#c49c94', '#f7b6d2', '#c7c7c7', '#dbdb8d', '#9edae5'
]

for index, feature in enumerate(districts_geojson['features']):
    # Obtener y limpiar nombre del distrito
    raw_name = feature['properties']['name']
    cleaned_name = clean_district_name(raw_name)
    
    # Asignar color de la paleta o generar uno si hay más distritos que colores
    district_color = COLOR_PALETTE[index] if index < len(COLOR_PALETTE) else f"#{random.randint(0x808080, 0xFFFFFF):06x}"
    
    # Guardar en estructuras
    district_colors[cleaned_name] = district_color
    districts.append({
        "name": cleaned_name,
        "color": district_color,
        "original_name": raw_name  # Mantener referencia al nombre original
    })

# Añadir color para distritos desconocidos
district_colors['Desconocido'] = '#cccccc'
districts.append({
    "name": 'Desconocido',
    "color": '#cccccc',
    "original_name": 'Distrito no identificado'
})

## 6. HTML Template Configuration

Creates the interactive map structure using Leaflet.js
- Base map layers
- Control panel
- Interactive elements
- Statistics display

In [7]:
html_template = Template('''<!DOCTYPE html>
<html lang="es">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Madrid Traffic Infrastructure Map</title>
    <link rel="stylesheet" href="https://unpkg.com/leaflet@1.7.1/dist/leaflet.css" />
    <link rel="stylesheet" href="styles.css">
    <link rel="stylesheet" href="https://unpkg.com/leaflet.markercluster@1.4.1/dist/MarkerCluster.Default.css">
    <link href="https://fonts.googleapis.com/css2?family=Nunito:wght@400;600&display=swap" rel="stylesheet">
</head>
<body>
    <div id="map"></div>
    <div class="control-panel">
        <button class="toggle-panel" onclick="toggleControlPanel()">☰</button>
        <div class="panel-content">
            <div class="filter-group">
                <h3>Categories</h3>
                <div id="category-filters"></div>
            </div>
            <div class="filter-group">
                <h3>Districts</h3>
                <div class="district-list" id="district-filters"></div>
            </div>
            <div class="stats-container">
                <h3>Statistics</h3>
                <div class="stats-item" id="total-count">Loading data...</div>
            </div>
        </div>
    </div>

    <script src="https://unpkg.com/leaflet@1.7.1/dist/leaflet.js"></script>
    <script src="https://unpkg.com/leaflet.markercluster@1.4.1/dist/leaflet.markercluster.js"></script>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/PapaParse/5.3.0/papaparse.min.js"></script>

    <script>
    // GitHub raw content base URL
    const BASE_URL = 'https://raw.githubusercontent.com/JSebastianIEU/madrid_traffic_map/main/'
    
    // Map initialization
    const map = L.map('map').setView([40.4168, -3.7038], 12);
    L.tileLayer('https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png').addTo(map);

    // Marker cluster group
    const markers = L.markerClusterGroup({
        chunkedLoading: true,
        chunkInterval: 100
    });
    map.addLayer(markers);

    // Data configuration
    const datasets = {
        'Traffic Lights': 'trafic.csv',
        'Streetlights': 'lamps.csv',
        'Acoustic Signals': 'acustic.csv'
    };

    // Load and process data
    async function loadData() {
        let allData = [];
        
        for (const [category, file] of Object.entries(datasets)) {
            const response = await fetch(`${BASE_URL}${file}`);
            const csvData = await response.text();
            
            const results = Papa.parse(csvData, {
                header: true,
                dynamicTyping: true,
                skipEmptyLines: true
            });
            
            const features = results.data.map(row => ({
                type: 'Feature',
                geometry: {
                    type: 'Point',
                    coordinates: [row.Longitude, row.Latitude]
                },
                properties: {
                    category: category,
                    district: row.district || 'Unknown',
                    type: row.type || 'N/A',
                    installation_date: row.installation_date || 'Unknown'
                }
            }));
            
            allData = allData.concat(features);
        }

        // Add markers to map
        L.geoJSON(allData, {
            pointToLayer: (feature, latlng) => {
                return L.marker(latlng, {
                    icon: L.icon({
                        iconUrl: `${BASE_URL}icons/${feature.properties.category.toLowerCase().replace(' ', '-')}.png`,
                        iconSize: [28, 28]
                    })
                }).bindPopup(this.createPopupContent(feature.properties));
            }
        }).addTo(markers);

        // Initialize filters and stats
        this.initializeFilters();
        this.updateStatistics();
    }

    function createPopupContent(properties) {
        return `
            <div class="popup-content">
                <h4>${properties.category}</h4>
                <p>District: ${properties.district}</p>
                ${properties.type ? `<p>Type: ${properties.type}</p>` : ''}
                ${properties.installation_date ? `<p>Installed: ${properties.installation_date}</p>` : ''}
            </div>
        `;
    }

    // Rest of the JavaScript functions (filters, stats) remain similar
    // but need to be adapted to work with the marker cluster group
    
    // Initialize the map
    loadData().catch(error => {
        console.error('Error loading data:', error);
        document.getElementById('total-count').textContent = 'Error loading data';
    });
    </script>
</body>
</html>
''')

# CSS Template with Improved Styling
css_template = Template('''
/* Base Styles */
* {
    margin: 0;
    padding: 0;
    box-sizing: border-box;
    font-family: 'Nunito', sans-serif;
}

body {
    background: #f0f2f5;
}

#map {
    height: 100vh;
    width: 100%;
    z-index: 1;
}

/* Control Panel */
.control-panel {
    position: fixed;
    top: 20px;
    right: 20px;
    width: 320px;
    background: rgba(255, 255, 255, 0.95);
    border-radius: 12px;
    box-shadow: 0 4px 24px rgba(0,0,0,0.1);
    backdrop-filter: blur(8px);
    z-index: 1000;
    transition: transform 0.3s ease;
}

.control-panel.collapsed {
    transform: translateX(calc(100% - 40px));
}

.toggle-panel {
    position: absolute;
    left: -40px;
    top: 10px;
    width: 40px;
    height: 40px;
    border: none;
    background: white;
    border-radius: 8px 0 0 8px;
    box-shadow: -4px 4px 12px rgba(0,0,0,0.1);
    cursor: pointer;
    font-size: 1.2rem;
}

.panel-content {
    padding: 20px;
    max-height: 90vh;
    overflow-y: auto;
}

/* Filter Groups */
.filter-group {
    margin-bottom: 1.5rem;
}

.filter-group h3 {
    font-size: 1rem;
    color: #2d3436;
    margin-bottom: 0.75rem;
    font-weight: 700;
}

.filter-item {
    display: flex;
    align-items: center;
    padding: 8px 12px;
    margin: 4px 0;
    background: rgba(0,0,0,0.03);
    border-radius: 6px;
    transition: background 0.2s;
}

.filter-item:hover {
    background: rgba(0,0,0,0.05);
}

.filter-item input {
    margin-right: 8px;
    accent-color: #0984e3;
}

/* District Colors */
.district-color {
    display: inline-block;
    width: 16px;
    height: 16px;
    border-radius: 4px;
    margin-right: 8px;
    box-shadow: 0 2px 4px rgba(0,0,0,0.1);
}

/* Statistics */
.stats-container {
    margin-top: 1.5rem;
    padding-top: 1.5rem;
    border-top: 1px solid rgba(0,0,0,0.1);
}

.stats-item {
    display: flex;
    justify-content: space-between;
    align-items: center;
    padding: 8px 12px;
    margin: 4px 0;
    background: rgba(0,0,0,0.03);
    border-radius: 6px;
}

.stat-item span:first-child {
    flex-grow: 1;
    margin-right: 1rem;
}

/* Popup Styling */
.leaflet-popup-content {
    font-size: 0.9rem;
    min-width: 200px;
}

.leaflet-popup-content h4 {
    color: #2d3436;
    margin-bottom: 0.5rem;
}

.leaflet-popup-content p {
    margin: 0.25rem 0;
    color: #636e72;
}
''')

# Generate Files
with open("index.html", "w", encoding="utf-8") as f:
    f.write(html_template.render(
        datasets=datasets,  # Now only contains 3 categories
        districts=districts,
        districts_geojson=districts_geojson,
        district_colors=district_colors
    ))

with open("styles.css", "w", encoding="utf-8") as f:
    f.write(css_template.render())

print("Files generated successfully:")
print("1. madrid_traffic_map.html")
print("2. styles.css")
print("\nMake sure you have:")
print("- Leaflet icons in an 'icons' folder")
print("- Madrid-districts.geojson file")
print("- All CSV datasets in the same directory")

Files generated successfully:
1. madrid_traffic_map.html
2. styles.css

Make sure you have:
- Leaflet icons in an 'icons' folder
- Madrid-districts.geojson file
- All CSV datasets in the same directory


In [8]:
#import csv trafic.csv, acustic.csv, lamps.csv

trafic = pd.read_csv('trafic.csv')
acustic = pd.read_csv('acustic.csv')
lamps = pd.read_csv('lamps.csv')
print("head of lamps")
display(lamps.head())
print("head of trafic")
display(trafic.head())
print("head of acustic")
display(acustic.head())
print("info of lamps")
lamps.info()
print("info of trafic")
trafic.info()
print("info of acustic")
acustic.info()

head of lamps


,type,district,neighborhood,Latitude,Longitude,address
0,DESCARGA,Hortaleza,Pinar del Rey,40.474942,-3.641197,"CALLE DE LAS ERAS, 3"
1,DESCARGA,Hortaleza,Pinar del Rey,40.475573,-3.641650,"CALLE DE LAS ERAS, 10"
2,LED,Hortaleza,Pinar del Rey,40.475020,-3.640825,"PLAZA DE MAR DEL PLATA, 12"
3,LED,Hortaleza,Pinar del Rey,40.474709,-3.640947,"CALLE DEL MAR AMARILLO, 21"
4,LED,Hortaleza,Pinar del Rey,40.474621,-3.641163,"CALLE DEL MAR AMARILLO, 19"


head of trafic


,type,district,id,id_cruce,Longitude,Latitude
0,SEMAFORO 2/200 PEATON,Centro,37203,99,-3.705259,40.429605
1,SEMAFORO 2/100,Centro,41153,99,-3.706057,40.429034
2,SEMAFORO 2/200 PEATON,Centro,37211,99,-3.705837,40.428900
3,SEMAFORO 3/200,Centro,19750,99,-3.705690,40.429576
4,SEMAFORO 2/200 PEATON,Centro,37212,99,-3.705839,40.428891


head of acustic


,type,district,id,id_cruce,Longitude,Latitude
0,AVISADOR ACUSTICO (MODERNO - DIGITAL PROGRAMAB...,Centro,3442,99,-3.705518,40.430051
1,AVISADOR ACUSTICO (MODERNO - DIGITAL PROGRAMAB...,Centro,3444,99,-3.705245,40.429411
2,AVISADOR ACUSTICO (MODERNO - DIGITAL PROGRAMAB...,Centro,3439,99,-3.706356,40.429531
3,AVISADOR ACUSTICO (MODERNO - DIGITAL PROGRAMAB...,Centro,3443,99,-3.705267,40.429625
4,AVISADOR ACUSTICO (MODERNO - DIGITAL PROGRAMAB...,Centro,3445,99,-3.705836,40.429176


info of lamps
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 233309 entries, 0 to 233308
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   type          233309 non-null  object 
 1   district      233309 non-null  object 
 2   neighborhood  233309 non-null  object 
 3   Latitude      233309 non-null  float64
 4   Longitude     233309 non-null  float64
 5   address       224704 non-null  object 
dtypes: float64(2), object(4)
memory usage: 10.7+ MB
info of trafic
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57073 entries, 0 to 57072
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   type       57073 non-null  object 
 1   district   57073 non-null  object 
 2   id         57073 non-null  int64  
 3   id_cruce   57073 non-null  int64  
 4   Longitude  57073 non-null  float64
 5   Latitude   57073 non-null  float64
dtypes: float64(2), int64(2)

In [7]:
# code to remove all empty rows in the dataframes where the latitude and longitude are empty and replace the csv directly
lamps = lamps.dropna(subset=['Latitude', 'Longitude'])
lamps.to_csv('lamps.csv', index=False)
trafic = trafic.dropna(subset=['Latitude', 'Longitude'])
trafic.to_csv('trafic.csv', index=False)
acustic = acustic.dropna(subset=['Latitude', 'Longitude'])
acustic.to_csv('acustic.csv', index=False)